In [4]:
!pip install geocoder
!pip install folium
!pip install geopy
print('Packages installed.')

     |████████████████████████████████| 102kB 10.4MB/s ta 0:00:01
     |████████████████████████████████| 102kB 10.4MB/s ta 0:00:01
Packages installed.


In [5]:
!pip install BeautifulSoup4

In [6]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
from geopy.geocoders import Nominatim
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [16]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [17]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [18]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,Parkwoods,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
5,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN
7,M8A,Not assigned,Not assigned,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",NaN,NaN
9,M1B,Scarborough,"Malvern, Rouge",NaN,NaN


In [19]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,Parkwoods,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
5,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN
7,M8A,Not assigned,Not assigned,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",NaN,NaN
9,M1B,Scarborough,"Malvern, Rouge",NaN,NaN


In [33]:
df_geo_coor = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df_toronto = pd.merge(neighbors, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN,NaN,NaN
2,M3A,North York,Parkwoods,NaN,NaN,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN,NaN,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN,NaN,NaN


In [36]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto